In [111]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-12-02 08:34:13--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [    <=>             ] 999,85K   985KB/s    in 1,0s    

2024-12-02 08:34:18 (985 KB/s) - ‘bank+marketing.zip’ saved [1023843]



In [3]:
data = pd.read_csv(r'/Users/hoang.hai.pham/Documents/code/Tutorials/machine-learning-zoomcamp/03-classification/notebooks/bank+marketing/bank/bank-full.csv', delimiter=';')

In [4]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


For the rest of the homework, you'll need to use only these columns:

age, job, marital, education,

balance, housing, contact, day,

month, duration, campaign, pdays,

previous, poutcome, y

In [5]:
keep_cols = [
    'age', 'job', 'marital', 'education',
    'balance', 'housing', 'contact', 'day',
    'month', 'duration', 'campaign', 'pdays',
    'previous', 'poutcome', 'y'
]

data = data[keep_cols]
data.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   balance    45211 non-null  int64 
 5   housing    45211 non-null  object
 6   contact    45211 non-null  object
 7   day        45211 non-null  int64 
 8   month      45211 non-null  object
 9   duration   45211 non-null  int64 
 10  campaign   45211 non-null  int64 
 11  pdays      45211 non-null  int64 
 12  previous   45211 non-null  int64 
 13  poutcome   45211 non-null  object
 14  y          45211 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.2+ MB


In [7]:
data.nunique()

age            77
job            12
marital         3
education       4
balance      7168
housing         2
contact         3
day            31
month          12
duration     1573
campaign       48
pdays         559
previous       41
poutcome        4
y               2
dtype: int64

In [6]:
data.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

# Q1

In [33]:
data[['education']].value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

# Q2

In [39]:
corr_cols = ['age', 'day', 'balance','campaign', 'pdays', 'previous']
rs = np.random.RandomState(0)
corr = data[corr_cols].corr()
corr.style.background_gradient(cmap='coolwarm').format(precision=2)

,age,day,balance,campaign,pdays,previous
age,1.00,-0.01,0.10,0.00,-0.02,0.00
day,-0.01,1.00,0.00,0.16,-0.09,-0.05
balance,0.10,0.00,1.00,-0.01,0.00,0.02
campaign,0.00,0.16,-0.01,1.00,-0.09,-0.03
pdays,-0.02,-0.09,0.00,-0.09,1.00,0.45
previous,0.00,-0.05,0.02,-0.03,0.45,1.00


In [42]:
corr.unstack().sort_values(ascending=False)

age       age         1.000000
day       day         1.000000
pdays     pdays       1.000000
campaign  campaign    1.000000
balance   balance     1.000000
previous  previous    1.000000
          pdays       0.454820
pdays     previous    0.454820
day       campaign    0.162490
campaign  day         0.162490
balance   age         0.097783
age       balance     0.097783
balance   previous    0.016674
previous  balance     0.016674
age       campaign    0.004760
campaign  age         0.004760
balance   day         0.004503
day       balance     0.004503
balance   pdays       0.003435
pdays     balance     0.003435
age       previous    0.001288
previous  age         0.001288
age       day        -0.009120
day       age        -0.009120
campaign  balance    -0.014578
balance   campaign   -0.014578
pdays     age        -0.023758
age       pdays      -0.023758
campaign  previous   -0.032855
previous  campaign   -0.032855
day       previous   -0.051710
previous  day        -0.051710
campaign

# Q3

In [46]:
data['y'] = (data['y'] == 'yes').astype(int)
data.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


In [48]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [54]:
# Split training/testing dataset
# set random_state to get specific random numbers when re-run cel
SEED = 42
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=SEED)
len(df_full_train), len(df_test)

(36168, 9043)

In [55]:
# df_full_train is 80% of the dataset
# To get 20% of the dataset => get 25% from df_full_train
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=SEED)
len(df_train), len(df_val)

(27126, 9042)

In [56]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train['y'].values
y_val = df_val['y'].values
y_test = df_test['y'].values

del df_train['y']
del df_val['y']
del df_test['y']

In [73]:
df_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome
0,32,technician,single,tertiary,1100,yes,cellular,11,aug,67,1,-1,0,unknown
1,38,entrepreneur,married,secondary,0,yes,cellular,17,nov,258,1,-1,0,unknown
2,49,blue-collar,married,secondary,3309,yes,cellular,15,may,349,2,-1,0,unknown
3,37,housemaid,married,primary,2410,no,cellular,4,aug,315,1,-1,0,unknown
4,31,self-employed,married,tertiary,3220,no,cellular,26,aug,74,4,-1,0,unknown


In [74]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, y_train)

In [77]:
other_cats = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
mi = df_train[other_cats].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

poutcome     0.029533
month        0.025090
contact      0.013356
housing      0.010343
job          0.007316
education    0.002697
marital      0.002050
dtype: float64

# Q4

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [78]:
df_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome
0,32,technician,single,tertiary,1100,yes,cellular,11,aug,67,1,-1,0,unknown
1,38,entrepreneur,married,secondary,0,yes,cellular,17,nov,258,1,-1,0,unknown
2,49,blue-collar,married,secondary,3309,yes,cellular,15,may,349,2,-1,0,unknown
3,37,housemaid,married,primary,2410,no,cellular,4,aug,315,1,-1,0,unknown
4,31,self-employed,married,tertiary,3220,no,cellular,26,aug,74,4,-1,0,unknown


In [80]:
train_dicts = df_train.to_dict(orient='records')

In [83]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [84]:
X_train.shape

(27126, 47)

In [88]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [134]:
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val.shape

(9042, 46)

In [135]:
# SOFT PREDICTIONS
# Probability of 0| Probability of 1
# Since we interested in probability of 1
# => use the right columns
y_pred = model.predict_proba(X_val)[:, 1]

In [136]:
y_pred

array([0.01253608, 0.00984399, 0.15347909, ..., 0.05488677, 0.0091198 ,
       0.28692033])

In [137]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = (y_pred >= 0.5).astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [138]:
original_score = df_pred.correct.mean()
original_score

0.9010174740101747

# Q5

In [124]:
features = df_train.columns.to_list()
drop_col = 'age'
features


['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [178]:
features = df_train.columns.to_list()

result_list = []
for drop_col in features:
    print(f'Drop feature: {drop_col}')

    dv = DictVectorizer(sparse=False)

    train_dicts = df_train.drop(columns=drop_col).to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    val_dicts = df_val.drop(columns=drop_col).to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)

    diff = original_score - score

    result_list.append({
        'feature_drop': drop_col, 
        'accuracy': score, 
        'diff': diff
    })

Drop feature: age
Drop feature: job
Drop feature: marital
Drop feature: education
Drop feature: balance
Drop feature: housing
Drop feature: contact
Drop feature: day
Drop feature: month
Drop feature: duration
Drop feature: campaign
Drop feature: pdays
Drop feature: previous
Drop feature: poutcome


In [183]:
df_q5 = pd.DataFrame(result_list)
df_q5.sort_values('diff', ascending=False)

,feature_drop,accuracy,diff
9,duration,0.889073,0.011944
13,poutcome,0.893939,0.007078
8,month,0.899801,0.001217
10,campaign,0.900133,0.000885
6,contact,0.900464,0.000553
2,marital,0.900575,0.000442
1,job,0.900796,0.000221
3,education,0.900907,0.000111
0,age,0.901017,0.000000
4,balance,0.901017,0.000000


In [192]:
df_q5[df_q5.index == df_q5['diff'].abs().idxmin()]

,feature_drop,accuracy,diff
0,age,0.901017,0.0


# Q6

In [200]:
c_list = [0.01, 0.1, 1, 10, 100]

result_list = []
for c in c_list:
    dv = DictVectorizer(sparse=False)

    train_dicts = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)

    diff = original_score - score

    result_list.append({
        'c': c, 
        'accuracy': round(score, 3), 
        'diff': diff
    })
    print(f"c: {c} - accuracy: {score}")

c: 0.01 - accuracy: 0.8976996239769962
c: 0.1 - accuracy: 0.9007962840079629
c: 1 - accuracy: 0.9007962840079629
c: 10 - accuracy: 0.9012386640123866
c: 100 - accuracy: 0.9010174740101747


In [202]:
df_q6 = pd.DataFrame(result_list)
df_q6.sort_values('accuracy', ascending=False)

,c,accuracy,diff
1,0.10,0.901,0.000221
2,1.00,0.901,0.000221
3,10.00,0.901,-0.000221
4,100.00,0.901,0.000000
0,0.01,0.898,0.003318
